In [3]:
!pip install nlplot
!pip install tqdm
!pip install -U pandas

In [4]:
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import nlplot
import plotly
from plotly.subplots import make_subplots
from plotly.offline import iplot

In [10]:
class anaResearchAndDevelopment:
    def __init__(self):
         #dbパス設定
        self._dbPath = "D:/教育/Edinet/Template/EdinetData.db"
        print(self._dbPath)

        self.textmining()

    # Select文実行
    def executeSelectQuery(self, query : str)-> pd.DataFrame:
        conn = sqlite3.connect(self._dbPath)  
        df = pd.read_sql_query(sql=query, con=conn)
        df.reset_index()
        #close
        conn.close()
        return df

    ''''''
    def textmining(self):

        nlp = spacy.load("ja_ginza")
        df提出者Year = self.executeSelectQuery("Select [Year],[提出者業種] FROM [View提出者有価証券リスト] Group By [Year],[提出者業種]")
        for column_name, row提出者Year in df提出者Year.iterrows():
            提出者業種 = row提出者Year["提出者業種"]
            Year = row提出者Year["Year"]
            df出力内容 = self.executeSelectQuery("Select [DocID],[出力項目] FROM [ViewAna研究開発費] where [Year] = " + str(Year) + " AND 提出者業種 like '" + 提出者業種 + "'")
            words_list = []
            bAdd = False
            for story in tqdm(df出力内容["出力項目"]):    
                story = story.replace('\n', '') 
                story = story.replace(' ', '')
                story = story.replace('\xa0', '')
                doc = nlp(story)
                words = []
                for token in doc:
                    if token.tag_ in ["名詞-普通名詞-一般","動詞-一般","形容詞-一般"]:
                        words.append(token.lemma_)
                        bAdd = True
                
                words_list.append(" ".join(words))  
            if not bAdd:
                continue

            df出力内容["words"] = pd.NA
            vectorizer = TfidfVectorizer()
            X = vectorizer.fit_transform(words_list)
            words = vectorizer.get_feature_names()
            for id, vec in zip(range(len(words_list)), X.toarray()):
                feature_words = []
                for w_id, tfidf in sorted(enumerate(vec), key=lambda x: x[1], reverse=True):
                    if tfidf < 0.1:
                        break
                    feature_word = words[w_id]
                    feature_words.append(feature_word)  
                df出力内容.at[id,"words"] = feature_words
            df出力内容 = df出力内容.reset_index()

            npt = nlplot.NLPlot(df出力内容, target_col='words')
            stopwords = npt.get_stopword(top_n=0, min_freq=0)+ ["当社","行う","おく", "開発費", "費用", "事業","研究開発", "こと", "よる", "会計年度", "関する", "セグメント", "進める", "事業年度", "向ける", "グループ" ]  

            '''gram = npt.bar_ngram(
                title='uni-gram',
                xaxis_label='word_count',
                yaxis_label='word',
                ngram=1,
                top_n=50,
                width=800,
                height=1100,
                color=None,
                horizon=True,
                stopwords=stopwords,
                verbose=False,
                save=False,
            )
            gram.write_image(提出者業種 + "_" + str(Year) + '_gram.png')'''

            bGraph = False
            try:
                npt.build_graph(stopwords=stopwords, min_edge_frequency=3)
                bGraph = True
            except:
                print("Err")
            
            if not bGraph:
                continue
                
            co_network = npt.co_network(
                title=提出者業種 + "_" + str(Year),
                sizing=100,
                node_size='adjacency_frequency',
                color_palette='hls',
                width=1100,
                height=700,
                save=True
            )
            iplot(co_network)

anaResearchAndDevelopment()

D:/教育/Edinet/Template/EdinetData.db


0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 3/3 [00:00<00:00,  4.89it/s]


Err


100%|██████████| 11/11 [00:01<00:00,  9.70it/s]


Err


...

Err


100%|██████████| 6/6 [00:00<00:00, 13.70it/s]


Err


100%|██████████| 9/9 [00:01<00:00,  5.29it/s]


Err
